In [102]:
import pandas as pd
import numpy as np

In [103]:
data = pd.read_parquet("./data/clean_data_merged_v1.parquet")
data['HomeTeamName'] = data['HomeTeamName'].str.strip()
data['AwayTeamName'] = data['AwayTeamName'].str.strip()

#data = data[data['date'].dt.year >= 1980].copy()
data['year'] = data['date'].dt.year

In [104]:
data.head(5)

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg,year
0,1960-07-06 20:00:00,France,Yugoslavia,4,5,-1.0,-1,1960
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3,-1.0,-3,1960
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0,1.0,2,1960
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1,1.0,1,1960
4,1964-06-17 20:00:00,Spain,Hungary,2,1,1.0,1,1964


In [105]:
data['won'] = (data['HomeTeamGoals'] > data['AwayTeamGoals']).astype(int)
data['lost'] = (data['HomeTeamGoals'] < data['AwayTeamGoals']).astype(int)
data['tie'] = (data['HomeTeamGoals'] == data['AwayTeamGoals']).astype(int)


In [106]:
gen_stats = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_stats.columns = ['n_games', 'home_goals', 'away_goals', 'won', 'lost', 'tie']

In [107]:
gen_statsh = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsh.columns = ['n_games', 'home_goals_scored', 'away_goals_suffered', 'won', 'lost', 'tie']
gen_statsh = gen_statsh.reset_index()
gen_statsh

,HomeTeamName,year,n_games,home_goals_scored,away_goals_suffered,won,lost,tie
0,Albania,2016,1,0,1,0,1,0
1,Austria,2008,3,1,3,0,2,1
2,Austria,2016,1,0,2,0,1,0
3,Belgium,1972,1,1,2,0,1,0
4,Belgium,1980,3,4,4,1,1,1
...,...,...,...,...,...,...,...,...
141,West Germany,1980,1,3,2,1,0,0
142,West Germany,1984,3,2,2,1,1,1
143,West Germany,1988,4,6,3,2,1,1
144,Yugoslavia,1968,1,1,0,1,0,0


In [108]:
gen_statsh = data.groupby(["HomeTeamName", "year"]).agg({"HomeTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsh.columns = ['n_games', 'scored_goals', 'suffered_goals', 'won', 'lost', 'tie']
gen_statsh.index = gen_statsh.index.rename(['team', 'year'])


gen_statsa = data.groupby(["AwayTeamName", "year"]).agg({"AwayTeamName": "size",
                                                       "HomeTeamGoals": "sum",
                                                       "AwayTeamGoals": "sum",
                                                       "won": "sum",
                                                       "lost": "sum",
                                                       "tie": "sum"})
gen_statsa.columns = ['n_games', 'suffered_goals', 'scored_goals', 'won', 'lost', 'tie']
gen_statsa.index = gen_statsa.index.rename(['team', 'year'])

gen_stats = gen_statsh.add(gen_statsa, fill_value=0).reset_index()
gen_stats.sort_values("year", inplace=True)


gen_stats['year'] += 1

# stats um time contra o outro

In [109]:
gen_stats

,team,year,lost,n_games,scored_goals,suffered_goals,tie,won
152,Yugoslavia,1961,1.0,2.0,6.0,6.0,0.0,1.0
115,Soviet Union,1961,1.0,2.0,5.0,1.0,0.0,1.0
43,France,1961,1.0,2.0,4.0,7.0,0.0,1.0
22,Czechoslovakia,1961,1.0,2.0,2.0,3.0,0.0,1.0
63,Hungary,1965,0.0,2.0,4.0,3.0,0.0,2.0
...,...,...,...,...,...,...,...,...
100,Republic of Ireland,2017,1.0,4.0,3.0,6.0,1.0,2.0
86,Northern Ireland,2017,2.0,4.0,2.0,3.0,0.0,2.0
97,Portugal,2017,1.0,7.0,9.0,5.0,4.0,2.0
51,France,2017,1.0,7.0,13.0,5.0,1.0,5.0


In [110]:
gen_stats_cumulative = gen_stats.groupby("team").cumsum()
gen_stats_cumulative['team'] = gen_stats['team']
gen_stats_cumulative['year'] = gen_stats['year']
gen_stats_cumulative.sort_values("year", inplace=True)
gen_stats_cumulative

,year,lost,n_games,scored_goals,suffered_goals,tie,won,team
152,1961,1.0,2.0,6.0,6.0,0.0,1.0,Yugoslavia
115,1961,1.0,2.0,5.0,1.0,0.0,1.0,Soviet Union
43,1961,1.0,2.0,4.0,7.0,0.0,1.0,France
22,1961,1.0,2.0,2.0,3.0,0.0,1.0,Czechoslovakia
63,1965,0.0,2.0,4.0,3.0,0.0,2.0,Hungary
...,...,...,...,...,...,...,...,...
139,2017,5.0,13.0,8.0,15.0,5.0,3.0,Switzerland
143,2017,7.0,15.0,13.0,22.0,2.0,6.0,Turkey
51,2017,11.0,39.0,62.0,44.0,9.0,19.0,France
129,2017,14.0,40.0,55.0,36.0,11.0,15.0,Spain


In [111]:
gen_stats[gen_stats['team'] == 'Portugal']

,team,year,lost,n_games,scored_goals,suffered_goals,tie,won
91,Portugal,1985,0.0,4.0,4.0,4.0,2.0,2.0
92,Portugal,1997,1.0,4.0,5.0,2.0,1.0,2.0
93,Portugal,2001,2.0,5.0,10.0,4.0,0.0,3.0
94,Portugal,2005,4.0,6.0,8.0,6.0,1.0,1.0
95,Portugal,2009,2.0,4.0,7.0,6.0,0.0,2.0
96,Portugal,2013,2.0,5.0,6.0,4.0,1.0,2.0
97,Portugal,2017,1.0,7.0,9.0,5.0,4.0,2.0


In [112]:
gen_stats_cumulative[gen_stats_cumulative['team'] == 'Portugal']

,year,lost,n_games,scored_goals,suffered_goals,tie,won,team
91,1985,0.0,4.0,4.0,4.0,2.0,2.0,Portugal
92,1997,1.0,8.0,9.0,6.0,3.0,4.0,Portugal
93,2001,3.0,13.0,19.0,10.0,3.0,7.0,Portugal
94,2005,7.0,19.0,27.0,16.0,4.0,8.0,Portugal
95,2009,9.0,23.0,34.0,22.0,4.0,10.0,Portugal
96,2013,11.0,28.0,40.0,26.0,5.0,12.0,Portugal
97,2017,12.0,35.0,49.0,31.0,9.0,14.0,Portugal


In [113]:
def fn_features(df, team_features, team_features_lasteuro):
    df_ = pd.merge_asof(df, team_features, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home', '_away'])
    df_ = pd.merge_asof(df_, team_features_lasteuro, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features_lasteuro, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home_lasteuro', '_away_lasteuro'])
    
    return df_

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [115]:
for year in [2004, 2008, 2012, 2016]:
    tr = data[data['year'] < year].copy()
    val = data[data['year'] == year].copy()
    
    tr = tr.drop(['won', 'lost', 'tie'], axis=1)
    val = val.drop(['won', 'lost', 'tie'], axis=1)
    
   
    selected_features = ['lost_home',
       'n_games_home', 'scored_goals_home', 'suffered_goals_home', 'tie_home',
       'won_home', 'lost_away', 'n_games_away',
       'scored_goals_away', 'suffered_goals_away', 'tie_away', 'won_away',
        'lost_home_lasteuro',
       'n_games_home_lasteuro', 'scored_goals_home_lasteuro',
       'suffered_goals_home_lasteuro', 'tie_home_lasteuro',
       'won_home_lasteuro', 'lost_away_lasteuro',
       'n_games_away_lasteuro', 'scored_goals_away_lasteuro',
       'suffered_goals_away_lasteuro', 'tie_away_lasteuro',
       'won_away_lasteuro']
    
    tr_ = fn_features(tr, gen_stats_cumulative, gen_stats)
    val_ = fn_features(val, gen_stats_cumulative, gen_stats)
    
    
    Xtr = tr_[selected_features].fillna(0)
    Xval = val_[selected_features].fillna(0)
    ytr = tr['y_classif']
    yval = val['y_classif']
    
    mdl = RandomForestClassifier(random_state=0, min_samples_leaf=21, n_estimators=300, n_jobs=6)
    mdl.fit(Xtr, ytr)
    
    p = mdl.predict_proba(Xval)
    
    
        
    baseline = ytr.value_counts(normalize=True)
    baseline.index = [2,0,1]
    baseline = np.ones((Xval.shape[0], 3)) * baseline.sort_index().values
    
    
    print("Ano: {} | Baseline Log Loss: {:.4f} | Baseline AUC: {:.4f} | Log Loss: {:.4f} | ROC AUC: {:.4f}\n".format(year, log_loss(yval, baseline),
            roc_auc_score(yval, baseline, multi_class='ovr'), log_loss(yval, p), roc_auc_score(yval,p, multi_class='ovr')))
    #break


Ano: 2004 | Baseline Log Loss: 1.1082 | Baseline AUC: 0.5000 | Log Loss: 1.1300 | ROC AUC: 0.4909

Ano: 2008 | Baseline Log Loss: 1.0736 | Baseline AUC: 0.5000 | Log Loss: 1.0439 | ROC AUC: 0.5480

Ano: 2012 | Baseline Log Loss: 1.0708 | Baseline AUC: 0.5000 | Log Loss: 1.0435 | ROC AUC: 0.5270

Ano: 2016 | Baseline Log Loss: 1.0898 | Baseline AUC: 0.5000 | Log Loss: 1.0897 | ROC AUC: 0.5303



# Dados novos de 2021

In [116]:
tabela = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2020")

In [121]:
times = tabela[3]['Team[A]'].str.extract(r"([\w\s]+)").squeeze().values

In [122]:
times

array(['Belgium', 'Italy', 'Russia', 'Poland', 'Ukraine', 'Spain',
       'France', 'Turkey', 'England', 'Czech Republic', 'Finland',
       'Sweden', 'Croatia', 'Austria', 'Netherlands', 'Germany',
       'Portugal', 'Switzerland', 'Denmark', 'Wales', 'North Macedonia',
       'Hungary', 'Slovakia', 'Scotland'], dtype=object)

In [123]:
from itertools import permutations

In [124]:
raw = pd.DataFrame(list(permutations(times,2)))
raw.columns = ['ht', 'at']
raw['year'] = 2021

In [128]:
gen_stats_cumulative.head()

,year,lost,n_games,scored_goals,suffered_goals,tie,won,team
152,1961,1.0,2.0,6.0,6.0,0.0,1.0,Yugoslavia
115,1961,1.0,2.0,5.0,1.0,0.0,1.0,Soviet Union
43,1961,1.0,2.0,4.0,7.0,0.0,1.0,France
22,1961,1.0,2.0,2.0,3.0,0.0,1.0,Czechoslovakia
63,1965,0.0,2.0,4.0,3.0,0.0,2.0,Hungary


In [ ]:
def fn_features(df, team_features, team_features_lasteuro):
    df_ = pd.merge_asof(df, team_features, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home', '_away'])
    df_ = pd.merge_asof(df_, team_features_lasteuro, left_by=['HomeTeamName'], right_by=['team'], on=['year'])
    df_ = pd.merge_asof(df_, team_features_lasteuro, left_by=['AwayTeamName'], right_by=['team'], on=['year'], suffixes=['_home_lasteuro', '_away_lasteuro'])
    
    return df_

In [80]:
df2021 = pd.merge_asof(raw, gen_stats_cumulative, left_by=['ht'], right_by=['team'], on=['year'])
df2021 = pd.merge_asof(df2021, gen_stats_cumulative, left_by=['at'], right_by=['team'], on=['year'], suffixes=['_home', '_away'])
df2021 = pd.merge_asof(df2021, gen_stats, left_by=['ht'], right_by=['team'], on=['year'])
df2021 = pd.merge_asof(df2021, gen_stats, left_by=['at'], right_by=['team'], on=['year'], suffixes=['_home_lasteuro', '_away_lasteuro'])
df2021.head()

,ht,at,year,lost_home,n_games_home,scored_goals_home,suffered_goals_home,tie_home,won_home,team_home,...,suffered_goals_home_lasteuro,tie_home_lasteuro,won_home_lasteuro,team_away_lasteuro,lost_away_lasteuro,n_games_away_lasteuro,scored_goals_away_lasteuro,suffered_goals_away_lasteuro,tie_away_lasteuro,won_away_lasteuro
0,Belgium,Italy,2021,6.0,17.0,22.0,25.0,2.0,9.0,Belgium,...,5.0,0.0,2.0,Italy,2.0,5.0,6.0,2.0,1.0,2.0
1,Belgium,Russia,2021,6.0,17.0,22.0,25.0,2.0,9.0,Belgium,...,5.0,0.0,2.0,Russia,2.0,3.0,2.0,6.0,1.0,0.0
2,Belgium,Poland,2021,6.0,17.0,22.0,25.0,2.0,9.0,Belgium,...,5.0,0.0,2.0,Poland,1.0,5.0,4.0,2.0,3.0,1.0
3,Belgium,Ukraine,2021,6.0,17.0,22.0,25.0,2.0,9.0,Belgium,...,5.0,0.0,2.0,Ukraine,2.0,3.0,0.0,5.0,0.0,1.0
4,Belgium,Spain,2021,6.0,17.0,22.0,25.0,2.0,9.0,Belgium,...,5.0,0.0,2.0,Spain,0.0,4.0,5.0,4.0,0.0,4.0


In [133]:
tr = data.drop(['won', 'lost', 'tie'], axis=1)

selected_features = ['lost_home',
   'n_games_home', 'scored_goals_home', 'suffered_goals_home', 'tie_home',
   'won_home', 'lost_away', 'n_games_away',
   'scored_goals_away', 'suffered_goals_away', 'tie_away', 'won_away',
    'lost_home_lasteuro',
   'n_games_home_lasteuro', 'scored_goals_home_lasteuro',
   'suffered_goals_home_lasteuro', 'tie_home_lasteuro',
   'won_home_lasteuro', 'lost_away_lasteuro',
   'n_games_away_lasteuro', 'scored_goals_away_lasteuro',
   'suffered_goals_away_lasteuro', 'tie_away_lasteuro',
   'won_away_lasteuro']

tr_ = fn_features(tr, gen_stats_cumulative, gen_stats)
Xtr = tr_[selected_features].fillna(0)
ytr = tr['y_classif']

mdl = RandomForestClassifier(random_state=0, min_samples_leaf=21, n_estimators=300, n_jobs=6)
mdl.fit(Xtr, ytr)




RandomForestClassifier(min_samples_leaf=21, n_estimators=300, n_jobs=6,
                       random_state=0)

In [134]:
p = mdl.predict_proba(df2021[selected_features].fillna(0))

In [136]:
p[:, ::-1]

array([[0.26859506, 0.28423561, 0.44716933],
       [0.28902629, 0.26601573, 0.44495799],
       [0.35161884, 0.25494014, 0.39344102],
       ...,
       [0.50321154, 0.22454893, 0.27223954],
       [0.3163081 , 0.28723232, 0.39645957],
       [0.37737615, 0.2798526 , 0.34277125]])

In [137]:
df_p = pd.DataFrame(p[:, ::-1], columns=['home', 'tie', 'away'])
df_p['ht'] = df2021['ht']
df_p['at'] = df2021['at']
df_p

,home,tie,away,ht,at
0,0.268595,0.284236,0.447169,Belgium,Italy
1,0.289026,0.266016,0.444958,Belgium,Russia
2,0.351619,0.254940,0.393441,Belgium,Poland
3,0.436666,0.249513,0.313821,Belgium,Ukraine
4,0.282461,0.283628,0.433911,Belgium,Spain
...,...,...,...,...,...
547,0.227834,0.309755,0.462411,Scotland,Denmark
548,0.379319,0.258559,0.362122,Scotland,Wales
549,0.503212,0.224549,0.272240,Scotland,North Macedonia
550,0.316308,0.287232,0.396460,Scotland,Hungary


In [140]:
df_p[(df_p['ht'] == 'Ukraine') & (df_p['at'] == 'Sweden')]

,home,tie,away,ht,at
102,0.228505,0.284574,0.486921,Ukraine,Sweden
